In [1]:
!pip install tensorflow==2.12.0

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 8.1 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Un

In [1]:
!pip install pyngrok flask-ngrok

In [2]:
!ngrok authtoken 1vPKimF2WGxymKk9UAn8hokTwyk_3GXKRhPzJsULvRcXrV9Tz

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
import tensorflow as tf
import tensorflow_hub as hub

In [4]:
def load_model(model_path):
  """
  Loads a saved model from a specified path.
  """
  print(f"Loading saved model from: {model_path}")
  model = tf.keras.models.load_model(model_path,
                                     custom_objects={"KerasLayer":hub.KerasLayer})
  return model

In [5]:
# Load our model trained and class labels
loaded_models = load_model('/content/model.h5')

my_file = open("labels.txt", "r")
class_labels = my_file.read().split("\n")
my_file.close()

Loading saved model from: /content/model.h5


In [7]:
from flask import Flask, render_template, request, redirect, url_for
from PIL import Image, ImageOps
from keras.utils import image_utils
import os
import numpy as np
import io
import base64
from pyngrok import ngrok

# Assuming loaded_models and class_labels are already defined
IMAGE_SIZE = (224, 224)  # Adjust according to your model's input size
# loaded_models = your_model_loading_function()  # Replace with your actual model loading

# Snake information dictionary
snake_info_dict = {
    "Banded krait": {
        "description": "The Banded Krait is a highly venomous snake, easily recognized by its alternating yellow and black bands along the length of its body.",
        "habitat": "Primarily found in forests, agricultural areas, and near water sources across Southeast Asia.",
        "diet": "Feeds mainly on other snakes and occasionally small mammals and lizards.",
        "venomous": "Yes, it is highly venomous and can be fatal if untreated, though it is generally reluctant to bite.",
         "image_url": "/static/images/banded_krait.jpg"
    },
    "Daboia siamensis": {
        "description": "Also known as the Eastern Russell’s Viper, this snake is venomous and has a distinctive pattern of circular or oval spots along its thick, stocky body.",
        "habitat": "Found in grasslands, open fields, and farmlands throughout Southeast Asia.",
        "diet": "Primarily preys on rodents, but will also consume birds and small mammals.",
        "venomous": "Yes, it is highly venomous and known for its aggressive behavior when threatened.",
        "image_url": "/static/images/dabobia.jpg"
    },
    "Golden tree snake": {
        "description": "A mildly venomous, slender snake with green and black crossbands, accented by yellow coloration. Known for its ability to glide between trees.",
        "habitat": "Common in forests and suburban areas across Southeast Asia.",
        "diet": "Preys mostly on lizards, small birds, and occasionally bats.",
        "venomous": "Mildly venomous, not considered dangerous to humans.",
        "image_url": "/static/images/golden.jpg"
    },
    "King cobra": {
        "description": "The King Cobra is the longest venomous snake in the world, recognized by its hood display and ability to stand upright when threatened.",
        "habitat": "Inhabits dense forests, mangroves, and areas near water in South and Southeast Asia.",
        "diet": "Feeds primarily on other snakes, including venomous species, and occasionally small mammals.",
        "venomous": "Yes, it is highly venomous and can deliver a fatal bite if untreated.",
        "image_url": "/static/images/king.jpg"
    },
    "Kukri snake": {
        "description": "A small to medium-sized, non-venomous snake known for its curved teeth that resemble a Nepali kukri knife.",
        "habitat": "Inhabits forests, grasslands, and sometimes human settlements across Southeast Asia.",
        "diet": "Primarily feeds on eggs, but also consumes small lizards and invertebrates.",
        "venomous": "No, it is non-venomous and harmless to humans.",
        "image_url": "/static/images/kukri.jpg"
    },
    "Long-nosed whip snake": {
        "description": "An arboreal snake with a slender body, long snout, and green coloration that provides excellent camouflage in foliage.",
        "habitat": "Found in forests and bushes throughout South and Southeast Asia.",
        "diet": "Primarily feeds on lizards and frogs, using its precision and stealth to ambush prey.",
        "venomous": "Mildly venomous, not dangerous to humans.",
        "image_url": "/static/images/long.jpg"
    },
    "Malayan pit viper": {
        "description": "A highly venomous snake with a stout body, reddish-brown coloration, and distinctive dark triangular markings.",
        "habitat": "Typically found in forested areas and agricultural land across Southeast Asia.",
        "diet": "Feeds mainly on rodents, but also consumes frogs and small birds.",
        "venomous": "Yes, it is highly venomous and responsible for many bites due to its tendency to remain motionless when approached.",
        "image_url": "/static/images/malayan.jpg"
    },
    "Reticulated python": {
        "description": "One of the longest snakes in the world, known for its intricate and beautiful pattern of light and dark scales. It is non-venomous and very muscular.",
        "habitat": "Occupies a variety of habitats including forests, wetlands, and sometimes urban areas across Southeast Asia.",
        "diet": "Feeds on mammals and birds, often using its strength to constrict and subdue large prey.",
        "venomous": "No, it is non-venomous and poses no venomous threat to humans.",
        "image_url": "/static/images/reticulated.jpg"
    },
    "It's not snake": {
        "description": "This picture is not snake",
        "habitat": " - ",
        "diet": " - ",
        "venomous": " - ",
        "image_url": "/static/images/not.jpg"
    }
}


app = Flask(__name__)

IMAGE_SIZE = (224, 224)

def classify_snake(image):
    try:
        # Resize and preprocess the image
        image = ImageOps.fit(image, IMAGE_SIZE, Image.LANCZOS)
        x = image_utils.img_to_array(image) / 255.0
        x = np.expand_dims(x, axis=0)

        # Predict using the model
        preds = loaded_models.predict(x)
        pred_class = np.argmax(preds)
        confidence = np.max(preds) * 100

        # Return predicted class and confidence score (formatted to 2 decimal places)
        return class_labels[pred_class], f"{confidence:.2f}"
    except Exception as e:
        print(f"Error during classification: {e}")
        return None, None

@app.route('/')
def index():
    # Pass the snake_info_dict to the template
    return render_template('index.html', snake_info_dict=snake_info_dict)

@app.route('/upload', methods=['POST'])
def upload():
    image_data = request.form.get('image_data')

    if image_data:
        # Decode the base64 image data
        image_data = image_data.split(",")[1]  # Remove the base64 prefix
        image = Image.open(io.BytesIO(base64.b64decode(image_data))).convert('RGB')

        # Perform classification
        snake_name, confidence = classify_snake(image)
        snake_info = snake_info_dict.get(snake_name, {})

        # Save captured image for display
        image_folder = os.path.join('static', 'images')
        os.makedirs(image_folder, exist_ok=True)
        image_path = os.path.join(image_folder, 'captured_image.jpg')
        image.save(image_path)
        image_url = url_for('static', filename='images/captured_image.jpg')

        # Render the results page
        return render_template('result.html', image_url=image_url,
                               snake_name=snake_name,
                               confidence=confidence,
                               snake_info=snake_info)

    return redirect(url_for('index'))


if __name__ == "__main__":
    # Start ngrok to expose the app
    public_url = ngrok.connect(5000)
    print(f" * Running on {public_url}")

    # Run the Flask app
    app.run(host="0.0.0.0", port=5000)


 * Running on NgrokTunnel: "https://c51b-34-44-78-91.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [08/Nov/2024 07:12:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Nov/2024 07:12:57] "GET /static/images/banded_krait.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Nov/2024 07:12:57] "GET /static/images/dabobia.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Nov/2024 07:12:57] "GET /static/images/long.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Nov/2024 07:12:57] "GET /static/images/king.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Nov/2024 07:12:57] "GET /static/images/golden.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Nov/2024 07:12:57] "GET /static/images/kukri.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Nov/2024 07:12:57] "GET 

1/1 [==============================] - 0s 98ms/step


INFO:werkzeug:127.0.0.1 - - [08/Nov/2024 07:13:02] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Nov/2024 07:13:02] "GET /static/images/captured_image.jpg HTTP/1.1" 200 -
